# Transalating queries

In [1]:
%cd ..
from opensearchpy import OpenSearch, helpers
import pandas as pd
from pprint import pp
from dotenv import load_dotenv
from tqdm import tqdm
import os
def ppr(resp):
    pp(resp.raw)
import time

c:\ulb\SearchEngines_benchmark


c:\ulb\SearchEngines_benchmark\venv\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


### Connecting to database

In [2]:
load_dotenv()

# Recupera la variabile della password dall'ambiente
OPENSEARCH_INITIAL_ADMIN_PASSWORD = os.getenv("OPENSEARCH_INITIAL_ADMIN_PASSWORD")
OPENSEARCH_URL = os.getenv("OPENSEARCH_URL")  # Ad esempio http://localhost:9200

# Verifica che la password sia stata caricata correttamente
if OPENSEARCH_INITIAL_ADMIN_PASSWORD:
    print("Password caricata con successo.")
else:
    print("Errore nel caricare la password.")

Password caricata con successo.


In [3]:
client = OpenSearch(
    hosts=[OPENSEARCH_URL],
    http_auth=("admin", OPENSEARCH_INITIAL_ADMIN_PASSWORD),
    use_ssl=False,
    verify_certs=False,
    ssl_assert_hostname=False,
    ssl_show_warn=False
)
try:
    info = client.info()
    print("Conection to OpenSearch succeded!")
except Exception as e:
    print(f"Error when connecting: {e}")

Conection to OpenSearch succeded!


## Query translations

### Query 1

Search for comments that contain the word 'Python' and order them by their score.

Query:

```
SELECT TOP 20 id, Score
FROM comments
WHERE text LIKE '%python%'
ORDER BY Score DESC;
```


In [62]:
resp = client.search(
    index="comments",  # Replace with your actual index name
    body={
        "size": 20,  # Top 20 results
        "query": {
            "match": {
                "Text": "python"  # text containing 'python'
            }
        },
        "_source": ["id", "Score"],  # Select only id and Score fields
        "sort": [
            {"Score": {"order": "desc"}}  # Sort by Score in descending order
        ]
    }
)
resp

{'took': 113,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': [{'_index': 'comments',
    '_id': '976892',
    '_score': None,
    '_source': {'Score': 227},
    'sort': [227]},
   {'_index': 'comments',
    '_id': '1357893',
    '_score': None,
    '_source': {'Score': 152},
    'sort': [152]},
   {'_index': 'comments',
    '_id': '44302',
    '_score': None,
    '_source': {'Score': 149},
    'sort': [149]},
   {'_index': 'comments',
    '_id': '155130',
    '_score': None,
    '_source': {'Score': 127},
    'sort': [127]},
   {'_index': 'comments',
    '_id': '554974',
    '_score': None,
    '_source': {'Score': 116},
    'sort': [116]},
   {'_index': 'comments',
    '_id': '890284',
    '_score': None,
    '_source': {'Score': 114},
    'sort': [114]},
   {'_index': 'comments',
    '_id': '37041',
    '_score': None,
    '_source': {'Score': 111},
    '

### Query 2

Search for users that contain the word 'Python' AND 'SQL' in the AboutMe and order by their reputation and last access date

Query:

```
SELECT TOP 20 id, Score
FROM posts 
WHERE title LIKE '%python%' AND ClosedDate!=0
ORDER BY Score DESC;
```

In [59]:
resp = client.search(
    index="posts",  # Replace with your actual index name
    body={
        "size": 20,  # Top 20 results
        "query": {
            "bool": {
                "must": [
                    {"match": {"Title": "python"}}  # title containing 'python'
                ],
                "filter": [
                    {"exists": {"field": "ClosedDate"}},  # Ensure ClosedDate exists
                    {"range": {
                        "ClosedDate": {
                            "lt": '2070-01-01 00:00:00.000'  # ClosedDate should not be 0
                        }
                    }}
                ]
            }
        },
        "_source": ["id", "Score"],  # Select only id and Score fields
        "sort": [
            {"Score": {"order": "desc"}}  # Sort by Score in descending order
        ]
    }
)
resp

{'took': 2165,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 496, 'relation': 'eq'},
  'max_score': None,
  'hits': [{'_index': 'posts',
    '_id': '576169',
    '_score': None,
    '_source': {'Score': 1909},
    'sort': [1909]},
   {'_index': 'posts',
    '_id': '101754',
    '_score': None,
    '_source': {'Score': 1846},
    'sort': [1846]},
   {'_index': 'posts',
    '_id': '101268',
    '_score': None,
    '_source': {'Score': 1420},
    'sort': [1420]},
   {'_index': 'posts',
    '_id': '81584',
    '_score': None,
    '_source': {'Score': 1028},
    'sort': [1028]},
   {'_index': 'posts',
    '_id': '890128',
    '_score': None,
    '_source': {'Score': 781},
    'sort': [781]},
   {'_index': 'posts',
    '_id': '56011',
    '_score': None,
    '_source': {'Score': 718},
    'sort': [718]},
   {'_index': 'posts',
    '_id': '600268',
    '_score': None,
    '_source': {'Score': 601},
    'sort': [601]},
 

### Query 3

Search for users that contain the word 'Python' AND 'SQL' in the AboutMe and order by their reputation and last access date

Query:

```
SELECT TOP 20 AccountId, Reputation, LastAccessDate
FROM users
WHERE (AboutMe LIKE '%python%' AND AboutMe LIKE '%sql%') AND (AboutMe LIKE '%java%' OR AboutMe LIKE '%javascript%')
ORDER BY Reputation DESC, LastAccessDate DESC

```

In [60]:
resp = client.search(
    index="users",  # Replace with your actual index name
    body={
        "size": 20,  # Top 20 results
        "query": {
            "bool": {
                "must": [
                    {"match": {"AboutMe": "python"}},  # AboutMe containing 'python'
                    {"match": {"AboutMe": "sql"}}     # AboutMe containing 'sql'
                ],
                "should": [
                    {"match": {"AboutMe": "java"}},    # AboutMe containing 'java'
                    {"match": {"AboutMe": "javascript"}}  # AboutMe containing 'javascript'
                ],
                "minimum_should_match": 1  # At least one of the should conditions must match
            }
        },
        "_source": ["AccountId", "Reputation", "LastAccessDate"],  # Select only the required fields
        "sort": [
            {"Reputation": {"order": "desc"}},  # Sort by Reputation in descending order
            {"LastAccessDate": {"order": "desc"}}  # Sort by LastAccessDate in descending order
        ]
    }
)
resp

{'took': 21,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 143, 'relation': 'eq'},
  'max_score': None,
  'hits': [{'_index': 'users',
    '_id': '106224',
    '_score': None,
    '_source': {'Reputation': 483017,
     'LastAccessDate': '2018-06-03 04:38:12.047'},
    'sort': [483017, 1528000692047]},
   {'_index': 'users',
    '_id': '20860',
    '_score': None,
    '_source': {'Reputation': 352124,
     'LastAccessDate': '2018-06-03 04:16:35.617'},
    'sort': [352124, 1527999395617]},
   {'_index': 'users',
    '_id': '128421',
    '_score': None,
    '_source': {'Reputation': 129057,
     'LastAccessDate': '2018-01-31 17:58:19.627'},
    'sort': [129057, 1517421499627]},
   {'_index': 'users',
    '_id': '8815',
    '_score': None,
    '_source': {'Reputation': 90999,
     'LastAccessDate': '2018-05-10 18:54:13.467'},
    'sort': [90999, 1525978453467]},
   {'_index': 'users',
    '_id': '104184',
    '_score

### Query 4

Search for posts that contain the word 'Python' OR 'SQL' and that have more than 3 Favoirite counts

query:
```
SELECT TOP 20 id
FROM posts
WHERE (Body LIKE '%python%' OR Body LIKE '%sql%') AND FavoriteCount>3;
```

In [53]:
resp = client.search(
    index="posts",  # Replace with your actual index name
    body={
        "size": 20,  # Top 20 results
        "query": {
            "bool": {
                "must": [
                    {"range": {"FavoriteCount": {"gt": 3}}}  # Filter for FavoriteCount > 3
                ],
                "should": [
                    {"match": {"Body": "python"}},  # Body containing 'python'
                    {"match": {"Body": "sql"}}  # Body containing 'sql'
                ],
                "minimum_should_match": 1  # At least one of the should conditions must match
            }
        },
        "_source": ["id"]  # Select only the 'id' field
    }
)
resp

{'took': 86,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 6852, 'relation': 'eq'},
  'max_score': 11.963434,
  'hits': [{'_index': 'posts',
    '_id': '519753',
    '_score': 11.963434,
    '_source': {}},
   {'_index': 'posts', '_id': '956104', '_score': 11.557853, '_source': {}},
   {'_index': 'posts', '_id': '438185', '_score': 11.035779, '_source': {}},
   {'_index': 'posts', '_id': '1563967', '_score': 10.808755, '_source': {}},
   {'_index': 'posts', '_id': '853370', '_score': 10.301046, '_source': {}},
   {'_index': 'posts', '_id': '798180', '_score': 10.213999, '_source': {}},
   {'_index': 'posts', '_id': '1516249', '_score': 10.111643, '_source': {}},
   {'_index': 'posts', '_id': '1107297', '_score': 10.04074, '_source': {}},
   {'_index': 'posts', '_id': '1702906', '_score': 10.023988, '_source': {}},
   {'_index': 'posts', '_id': '16861', '_score': 9.940653, '_source': {}},
   {'_index': 'posts', '_

### Query 5

Search for comments that contain the word 'Python' BUT NOT 'SQL' created in 2008

Query:

```
SELECT TOP 20 id, CreationDate
FROM comments
WHERE text LIKE '%python%' AND text NOT LIKE '%sql%' AND CreationDate LIKE '2008%' ;
```


In [52]:
resp = client.search(
    index="comments",  # Replace with your actual index name
    body={
        "size": 20,  # Top 20 results
        "query": {
            "bool": {
                "must": [
                    {"match": {"Text": "python"}},  # Text containing 'python'
                ],
                "must_not": [
                    {"match": {"Text": "sql"}},  # Text not containing 'sql'
                ],
                "filter": [
                    {"range": {
                        "CreationDate": {
                            "gte": "2008-01-01 00:00:00.000",  # Start of 2008 with time
                            "lt": "2009-01-01 00:00:00.000"  # Start of 2009 with time
                        }
                    }}
                ]
            }
        },
        "_source": ["id", "CreationDate"]  # Select only id and CreationDate fields
    }
)
resp

{'took': 86,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 1066, 'relation': 'eq'},
  'max_score': 7.815918,
  'hits': [{'_index': 'comments',
    '_id': '218502',
    '_score': 7.815918,
    '_source': {'CreationDate': '2008-12-24 12:57:05.460'}},
   {'_index': 'comments',
    '_id': '185641',
    '_score': 7.723677,
    '_source': {'CreationDate': '2008-12-09 17:04:09.887'}},
   {'_index': 'comments',
    '_id': '120950',
    '_score': 7.631248,
    '_source': {'CreationDate': '2008-11-05 12:07:37.523'}},
   {'_index': 'comments',
    '_id': '219950',
    '_score': 7.538722,
    '_source': {'CreationDate': '2008-12-25 17:09:43.233'}},
   {'_index': 'comments',
    '_id': '171948',
    '_score': 7.5318804,
    '_source': {'CreationDate': '2008-12-02 20:44:43.597'}},
   {'_index': 'comments',
    '_id': '75946',
    '_score': 7.4073687,
    '_source': {'CreationDate': '2008-10-14 09:03:46.497'}},
   {'_index': 'c

### Query 6

Search for the users without downvotes that contain the word 'Python' OR 'SQL' in their AboutMe but double the value on the ranking of the word 'SQL'

```
SELECT id, DownVotes
FROM users 
WHERE MATCH('@AboutMe Python | SQL^2') and Downvotes=0
ORDER BY DownVotes DESC  

```

In [63]:
resp = client.search(
    index="users",  # Replace with your actual index name
    body={
        "query": {
            "bool": {
                "must": [
                    {
                        "bool": {
                            "should": [
                                {"match": {"AboutMe": "Python"}},  # Match 'Python' in 'AboutMe'
                                {"match": {"AboutMe": "SQL"}}     # Match 'SQL' in 'AboutMe'
                            ],
                            "minimum_should_match": 1  # Equivalent to 'OR' in Sphinx
                        }
                    }
                ],
                "filter": [
                    {"term": {"DownVotes": 0}}  # Filter for DownVotes=0
                ]
            }
        },
        "_source": ["id", "DownVotes"],  # Retrieve 'id' and 'DownVotes' fields
        "sort": [
            {"DownVotes": {"order": "desc"}}  # Sort by DownVotes in descending order
        ]
    }
)
resp

{'took': 12,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 907, 'relation': 'eq'},
  'max_score': None,
  'hits': [{'_index': 'users',
    '_id': '107036',
    '_score': None,
    '_source': {'DownVotes': 0},
    'sort': [0]},
   {'_index': 'users',
    '_id': '107094',
    '_score': None,
    '_source': {'DownVotes': 0},
    'sort': [0]},
   {'_index': 'users',
    '_id': '107107',
    '_score': None,
    '_source': {'DownVotes': 0},
    'sort': [0]},
   {'_index': 'users',
    '_id': '107127',
    '_score': None,
    '_source': {'DownVotes': 0},
    'sort': [0]},
   {'_index': 'users',
    '_id': '107368',
    '_score': None,
    '_source': {'DownVotes': 0},
    'sort': [0]},
   {'_index': 'users',
    '_id': '107511',
    '_score': None,
    '_source': {'DownVotes': 0},
    'sort': [0]},
   {'_index': 'users',
    '_id': '107560',
    '_score': None,
    '_source': {'DownVotes': 0},
    'sort': [0]},
   {'_ind

### Query 7

Search for posts that their body start with the <p>, their title ends with sql and they contain the word sql in their tags
DOESN'T WORK HAS PROBLEMS WITH WHITE SPACES

Query:

```
SELECT TOP 20 *
FROM posts 
WHERE body LIKE '<p>%' AND title LIKE '%sql' AND tags LIKE '%sql%';
```


In [38]:
# mapping = client.indices.get_mapping(index="posts")
# mapping

In [ ]:
resp = client.search(
    index="posts",
    body={
        "size": 20,  # Fetch the top 20 results
        "query": {
            "bool": {
                "must": [
                    {"match": {"Body": "<p>"}},
                    {"match": {"Title": "sql"}},
                    {"match": {"Tags":"sql"}}
                ]
            }
        },
        # Return specific fields
        "_source": ["Body", "Title", "Tags", "Score"]
    }
)
resp

{'took': 33,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': 9.925262,
  'hits': [{'_index': 'posts',
    '_id': '1508434',
    '_score': 9.925262,
    '_source': {'Title': 'Sql Competitions',
     'Body': '<p>Sorry to ask this kind of questions.</p>\n\n<p>But I am eager to know some good sites where I can have the opportunity to have some sql competitions.</p>\n\n<p>I found 1 \n<a href="http://www.experts-exchange.com/Microsoft/Development/MS-SQL-Server/Q_20623909.html" rel="nofollow noreferrer">TroyK\'s T-SQL Challenge</a></p>\n\n<p>but it is not free. </p>\n\n<p>I am looking for a free one.</p>\n\n<p>And also C# programming competitions. One I found is <a href="http://www.codechef.com/OCT09/" rel="nofollow noreferrer">Code Chef</a></p>\n\n<p>I am Looking for some more.</p>\n\n<p>Also, it should be for any ages!</p>\n\n<p>Edited:</p>\n\n<p>I liked <a href="http://projecte

### Query 8

Search for users  who have in their about me Python and SQL but if they have any additional programming lenguage rank them higher, and display the weight of the ranking.

Query:

```
SELECT id, WEIGHT()
FROM users
WHERE MATCH('@aboutme python sql MAYBE ( c++ || r || ruby || php || java || javascript)')  
```


In [65]:
resp = client.search(
    index="users",  # Replace with your actual index name
    body={
        "query": {
            "bool": {
                "must": [
                    {
                        "bool": {
                            "should": [
                                {"match": {"AboutMe": "python"}},  # Match 'python' in 'aboutme'
                                {"match": {"AboutMe": "sql"}},    # Match 'sql' in 'aboutme'
                                {
                                    "bool": {
                                        "should": [
                                            {"match": {"AboutMe": "c++"}},  # Match 'c++' in 'aboutme'
                                            {"match": {"AboutMe": "r"}},    # Match 'r' in 'aboutme'
                                            {"match": {"AboutMe": "ruby"}}, # Match 'ruby' in 'aboutme'
                                            {"match": {"AboutMe": "php"}},  # Match 'php' in 'aboutme'
                                            {"match": {"AboutMe": "java"}}, # Match 'java' in 'aboutme'
                                            {"match": {"AboutMe": "javascript"}} # Match 'javascript' in 'aboutme'
                                        ]
                                    }
                                }
                            ],
                            "minimum_should_match": 1  # At least one condition should match
                        }
                    }
                ]
            }
        },
        "_source": ["id"],  # Retrieve only 'id'
    }
)
resp

{'took': 39,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 9746, 'relation': 'eq'},
  'max_score': 18.46382,
  'hits': [{'_index': 'users',
    '_id': '220432',
    '_score': 18.46382,
    '_source': {}},
   {'_index': 'users', '_id': '58003', '_score': 18.258615, '_source': {}},
   {'_index': 'users', '_id': '230747', '_score': 17.879433, '_source': {}},
   {'_index': 'users', '_id': '100072', '_score': 17.774277, '_source': {}},
   {'_index': 'users', '_id': '5675', '_score': 17.596859, '_source': {}},
   {'_index': 'users', '_id': '151312', '_score': 17.566162, '_source': {}},
   {'_index': 'users', '_id': '120401', '_score': 17.08394, '_source': {}},
   {'_index': 'users', '_id': '185042', '_score': 16.82148, '_source': {}},
   {'_index': 'users', '_id': '166820', '_score': 16.714514, '_source': {}},
   {'_index': 'users', '_id': '168542', '_score': 16.663248, '_source': {}}]}}

### Query 9

Search for posts with python in the title, sql in the body and c++ in the tags and order them by score. 

Query:

```
SELECT TOP 20 id, Score
FROM posts 
WHERE title LIKE '%python%' AND body LIKE '%sql%' AND tags LIKE '%c++%'
ORDER BY Score DESC;
```


In [43]:
resp = client.search(
    index="posts",
    body={
        "size": 20,  # Retrieve the top 20 results
        "query": {
            "bool": {
                "must": [  # Match all conditions
                    {"match": {"Body": "sql"}},  # Body contains "sql"
                    {"match": {"Body": "python"}},  # Body contains "python"
                    {"match": {"Body": "c++"}}  # Body contains "c++"
                ]
            }
        },
        "sort": [
            {"CommentCount": {"order": "desc"}}  # Sort by CommentCount (or adjust to 'Score' if needed)
        ],
        "_source": ["Body", "CommentCount", "OwnerUserId"]  # Specify the fields to retrieve
    }
)

resp

{'took': 44,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 166, 'relation': 'eq'},
  'max_score': None,
  'hits': [{'_index': 'posts',
    '_id': '1043360',
    '_score': None,
    '_source': {'OwnerUserId': 10661,
     'CommentCount': 20,
     'Body': '<p>"how quickly you can code" so totally concerns me that I happily gave up the long, slow slog through getting things to compile.</p>\n\n<p>Advantages of dynamic languages.</p>\n\n<ol>\n<li><p>No compile, no build.  Just Code and Test followed by deploy to production.</p></li>\n<li><p>Immediate gratification.  No time spent wringing my hands over what an API call might be.  Just type it interactively in the Python >>> prompt and see what it <em>actually</em> does.</p></li>\n<li><p>Very, very short design cycles.  Rather than carefully crafting an class hierarchy with bonus interface definitions and proper abstract declarations and overrides, I can just code the c

### Query 10

Search for comments that have the words Python and Sql within a 3 word range in the text and order them by score

```
SELECT id, Score
FROM comments
WHERE MATCH('@text sql NEAR/3 python')
ORDER BY Score DESC
```


In [ ]:
resp = client.search(
    index="comments",  # Replace with your actual index name
    body={
        "query": {
            "bool": {
                "must": [
                    {
                        "match_phrase": {
                            "Text": {
                                "query": "sql python",  # Match 'sql' and 'python' in 'text'
                                "slop": 3  # Allow up to 3 positions between 'sql' and 'python'
                            }
                        }
                    }
                ]
            }
        },
        "_source": ["id", "Score"],  # Retrieve 'id' and 'Score'
        "sort": [
            {"Score": {"order": "desc"}}  # Sort by Score in descending order
        ]
    }
)
resp

{'took': 47,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 11, 'relation': 'eq'},
  'max_score': None,
  'hits': [{'_index': 'comments',
    '_id': '245274',
    '_score': None,
    '_source': {'Score': 4},
    'sort': [4]},
   {'_index': 'comments',
    '_id': '1103163',
    '_score': None,
    '_source': {'Score': 4},
    'sort': [4]},
   {'_index': 'comments',
    '_id': '1655438',
    '_score': None,
    '_source': {'Score': 1},
    'sort': [1]},
   {'_index': 'comments',
    '_id': '234029',
    '_score': None,
    '_source': {'Score': 0},
    'sort': [0]},
   {'_index': 'comments',
    '_id': '296164',
    '_score': None,
    '_source': {'Score': 0},
    'sort': [0]},
   {'_index': 'comments',
    '_id': '533254',
    '_score': None,
    '_source': {'Score': 0},
    'sort': [0]},
   {'_index': 'comments',
    '_id': '667097',
    '_score': None,
    '_source': {'Score': 0},
    'sort': [0]},
   {'_index': '

### Query 11

Search for posts that have any 2 lenguages of programmation and order them by score

```
SELECT TOP 20 id, Score
            FROM comments
            WHERE
            (text LIKE '%sql%' AND text LIKE '%python%') OR
            (text LIKE '%sql%' AND text LIKE '%c++%') OR
            (text LIKE '%sql%' AND text LIKE '%r%') OR
            (text LIKE '%sql%' AND text LIKE '%ruby%') OR
            (text LIKE '%sql%' AND text LIKE '%php%') OR
            (text LIKE '%sql%' AND text LIKE '%java%') OR
            (text LIKE '%sql%' AND text LIKE '%javascript%') OR
            (text LIKE '%python%' AND text LIKE '%c++%') OR
            (text LIKE '%python%' AND text LIKE '%r%') OR
            (text LIKE '%python%' AND text LIKE '%ruby%') OR
            (text LIKE '%python%' AND text LIKE '%php%') OR
            (text LIKE '%python%' AND text LIKE '%java%') OR
            (text LIKE '%python%' AND text LIKE '%javascript%') OR
            (text LIKE '%c++%' AND text LIKE '%r%') OR
            (text LIKE '%c++%' AND text LIKE '%ruby%') OR
            (text LIKE '%c++%' AND text LIKE '%php%') OR
            (text LIKE '%c++%' AND text LIKE '%java%') OR
            (text LIKE '%c++%' AND text LIKE '%javascript%') OR
            (text LIKE '%r%' AND text LIKE '%ruby%') OR
            (text LIKE '%r%' AND text LIKE '%php%') OR
            (text LIKE '%r%' AND text LIKE '%java%') OR
            (text LIKE '%r%' AND text LIKE '%javascript%') OR
            (text LIKE '%ruby%' AND text LIKE '%php%') OR
            (text LIKE '%ruby%' AND text LIKE '%java%') OR
            (text LIKE '%ruby%' AND text LIKE '%javascript%') OR
            (text LIKE '%php%' AND text LIKE '%java%') OR
            (text LIKE '%php%' AND text LIKE '%javascript%') OR
            (text LIKE '%java%' AND text LIKE '%javascript%')
            ORDER BY Score DESC;           
``` 

In [11]:
resp = client.search(
    index="comments",
    body={
        "size": 20,  # Retrieve top 20 results
        "query": {
            "bool": {
                "should": [
                    {"bool": {"must": [{"match": {"Text": "sql"}}, {"match": {"Text": "python"}}]}},
                    {"bool": {"must": [{"match": {"Text": "sql"}}, {"match": {"Text": "c++"}}]}},
                    {"bool": {"must": [{"match": {"Text": "sql"}}, {"match": {"Text": "r"}}]}},
                    {"bool": {"must": [{"match": {"Text": "sql"}}, {"match": {"Text": "ruby"}}]}},
                    {"bool": {"must": [{"match": {"Text": "sql"}}, {"match": {"Text": "php"}}]}},
                    {"bool": {"must": [{"match": {"Text": "sql"}}, {"match": {"Text": "java"}}]}},
                    {"bool": {"must": [{"match": {"Text": "sql"}}, {"match": {"Text": "javascript"}}]}},
                    {"bool": {"must": [{"match": {"Text": "python"}}, {"match": {"Text": "c++"}}]}},
                    {"bool": {"must": [{"match": {"Text": "python"}}, {"match": {"Text": "r"}}]}},
                    {"bool": {"must": [{"match": {"Text": "python"}}, {"match": {"Text": "ruby"}}]}},
                    {"bool": {"must": [{"match": {"Text": "python"}}, {"match": {"Text": "php"}}]}},
                    {"bool": {"must": [{"match": {"Text": "python"}}, {"match": {"Text": "java"}}]}},
                    {"bool": {"must": [{"match": {"Text": "python"}}, {"match": {"Text": "javascript"}}]}},
                    {"bool": {"must": [{"match": {"Text": "c++"}}, {"match": {"Text": "r"}}]}},
                    {"bool": {"must": [{"match": {"Text": "c++"}}, {"match": {"Text": "ruby"}}]}},
                    {"bool": {"must": [{"match": {"Text": "c++"}}, {"match": {"Text": "php"}}]}},
                    {"bool": {"must": [{"match": {"Text": "c++"}}, {"match": {"Text": "java"}}]}},
                    {"bool": {"must": [{"match": {"Text": "c++"}}, {"match": {"Text": "javascript"}}]}},
                    {"bool": {"must": [{"match": {"Text": "r"}}, {"match": {"Text": "ruby"}}]}},
                    {"bool": {"must": [{"match": {"Text": "r"}}, {"match": {"Text": "php"}}]}},
                    {"bool": {"must": [{"match": {"Text": "r"}}, {"match": {"Text": "java"}}]}},
                    {"bool": {"must": [{"match": {"Text": "r"}}, {"match": {"Text": "javascript"}}]}},
                    {"bool": {"must": [{"match": {"Text": "ruby"}}, {"match": {"Text": "php"}}]}},
                    {"bool": {"must": [{"match": {"Text": "ruby"}}, {"match": {"Text": "java"}}]}},
                    {"bool": {"must": [{"match": {"Text": "ruby"}}, {"match": {"Text": "javascript"}}]}},
                    {"bool": {"must": [{"match": {"Text": "php"}}, {"match": {"Text": "java"}}]}},
                    {"bool": {"must": [{"match": {"Text": "php"}}, {"match": {"Text": "javascript"}}]}},
                    {"bool": {"must": [{"match": {"Text": "java"}}, {"match": {"Text": "javascript"}}]}}
                ]
            }
        },
        "sort": [{"Score": {"order": "desc"}}],  # Sort by Score descending
        "_source": ["Id", "Score"]  # Retrieve only `Id` and `Score` fields
    }
)


### Query 12:

Search for users that have the word python or sql in the first 50 words of their AboutMe  and location is USA

```
SELECT id
FROM users 
WHERE MATCH('@AboutMe[50] python | sql @location USA') 
```
## FIX IT

In [68]:
resp = client.search(
    index="users",  # Replace with your actual index name
    body={
        "query": {
            "bool": {
                "must": [
                    {
                        "bool": {
                            "should": [
                                {"match": {"AboutMe": "python"}},  # Match 'python' in 'AboutMe'
                                {"match": {"AboutMe": "sql"}}  # Match 'sql' in 'AboutMe'
                            ]
                        }
                    },
                    {
                        "match": {"location": "USA"}  # Match 'USA' in 'location'
                    }
                ]
            }
        },
        "_source": ["id"]  # Retrieve only the 'id' field
    }
)
resp

{'took': 10,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

### Query 13:
Query that counts how many times are mentioned this programming languages on the posts (Sql, p.Python, R, Java, JavaScript, C++, Ruby, PHP)

### Post data

The query translated is the following:

```
SELECT 
    CASE
        WHEN Body LIKE '%sql%' THEN 'SQL'
        WHEN Body LIKE '%python%' THEN 'Python'
        WHEN Body LIKE '% R %' THEN 'R'
        WHEN Body LIKE '%java%' THEN 'Java'
        WHEN Body LIKE '%javascript%' THEN 'JavaScript'
        WHEN Body LIKE '%c++%' THEN 'C++'
        WHEN Body LIKE '%ruby%' THEN 'Ruby'
        WHEN Body LIKE '%php%' THEN 'PHP'
        ELSE 'Other'
    END AS Programming_Language,
    COUNT(*) AS Post_Count
FROM 
    posts
WHERE 
    Body LIKE '%sql%' 
    OR Body LIKE '%python%' 
    OR Body LIKE '% R %'
    OR Body LIKE '%java%' 
    OR Body LIKE '%javascript%' 
    OR Body LIKE '%c++%' 
    OR Body LIKE '%ruby%' 
    OR Body LIKE '%php%' 
GROUP BY 
    CASE
        WHEN Body LIKE '%sql%' THEN 'SQL'
        WHEN Body LIKE '%python%' THEN 'Python'
        WHEN Body LIKE '% R %' THEN 'R'
        WHEN Body LIKE '%java%' THEN 'Java'
        WHEN Body LIKE '%javascript%' THEN 'JavaScript'
        WHEN Body LIKE '%c++%' THEN 'C++'
        WHEN Body LIKE '%ruby%' THEN 'Ruby'
        WHEN Body LIKE '%php%' THEN 'PHP'
        ELSE 'Other'
    END
ORDER BY
    Post_Count;
```

In [45]:
# Measure execution time
start_time = time.time()
# Query that counts how many times are mentioned this programming languages on the posts (Sql, python, R, Java, JavaScript, C++, Ruby, PHP)
query1 = client.search(
    index='posts',
    body={
        "size": 0,  # We're aggregating, not fetching individual documents
        "query": {
            "bool": {
                "should": [
                    {"match_phrase": {"Body": "sql"}},
                    {"match_phrase": {"Body": "python"}},
                    {"match_phrase": {"Body": " R "}},
                    {"match_phrase": {"Body": "java"}},
                    {"match_phrase": {"Body": "javascript"}},
                    {"match_phrase": {"Body": "c++"}},
                    {"match_phrase": {"Body": "ruby"}},
                    {"match_phrase": {"Body": "php"}}
                ]
            }
        },
        "aggs": {
            "Programming_Language": {
                "filters": {
                    "filters": {
                        "SQL": {"match_phrase": {"Body": "sql"}},
                        "Python": {"match_phrase": {"Body": "python"}},
                        "R": {"match_phrase": {"Body": " R "}},
                        "Java": {"match_phrase": {"Body": "java"}},
                        "JavaScript": {"match_phrase": {"Body": "javascript"}},
                        "C++": {"match_phrase": {"Body": "c++"}},
                        "Ruby": {"match_phrase": {"Body": "ruby"}},
                        "PHP": {"match_phrase": {"Body": "php"}}
                    }
                }
            }
        }
    }
)
execution_time = time.time() - start_time
query1

{'took': 58,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 10000, 'relation': 'gte'},
  'max_score': None,
  'hits': []},
 'aggregations': {'Programming_Language': {'buckets': {'C++': {'doc_count': 158785},
    'Java': {'doc_count': 64767},
    'JavaScript': {'doc_count': 54241},
    'PHP': {'doc_count': 48633},
    'Python': {'doc_count': 36153},
    'R': {'doc_count': 21861},
    'Ruby': {'doc_count': 15349},
    'SQL': {'doc_count': 69452}}}}}

### Query 14
Query that counts how many times are mentioned this programming languages on comments (Sql, python, R, Java, JavaScript, C++, Ruby, PHP)

### Comments data
The query transalated is:
```
SELECT 
    CASE
        WHEN Text LIKE '%sql%' THEN 'SQL'
        WHEN Text LIKE '%python%' THEN 'Python'
        WHEN Text LIKE '% R %' THEN 'R'
        WHEN Text LIKE '%java%' THEN 'Java'
        WHEN Text LIKE '%javascript%' THEN 'JavaScript'
        WHEN Text LIKE '%c++%' THEN 'C++'
        WHEN Text LIKE '%ruby%' THEN 'Ruby'
        WHEN Text LIKE '%php%' THEN 'PHP'
        ELSE 'Other'
    END AS Programming_Language,
    COUNT(*) AS Post_Count
FROM 
    comments
WHERE 
    Text LIKE '%sql%' 
    OR Text LIKE '%python%' 
    OR Text LIKE '% R %'
    OR Text LIKE '%java%' 
    OR Text LIKE '%javascript%' 
    OR Text LIKE '%c++%' 
    OR Text LIKE '%ruby%' 
    OR Text LIKE '%php%' 
GROUP BY 
    CASE
        WHEN Text LIKE '%sql%' THEN 'SQL'
        WHEN Text LIKE '%python%' THEN 'Python'
        WHEN Text LIKE '% R %' THEN 'R'
        WHEN Text LIKE '%java%' THEN 'Java'
        WHEN Text LIKE '%javascript%' THEN 'JavaScript'
        WHEN Text LIKE '%c++%' THEN 'C++'
        WHEN Text LIKE '%ruby%' THEN 'Ruby'
        WHEN Text LIKE '%php%' THEN 'PHP'
        ELSE 'Other'
    END;
```

In [6]:

# Measure execution time
start_time = time.time()
# Query that counts how many times are mentioned this programming languages on the posts (Sql, python, R, Java, JavaScript, C++, Ruby, PHP)
query1 = client.search(
    index='comments',
    body={
        "size": 0,  # We're aggregating, not fetching individual documents
        "query": {
            "bool": {
                "should": [
                    {"match_phrase": {"Text": "sql"}},
                    {"match_phrase": {"Text": "python"}},
                    {"match_phrase": {"Text": " R "}},
                    {"match_phrase": {"Text": "java"}},
                    {"match_phrase": {"Text": "javascript"}},
                    {"match_phrase": {"Text": "c++"}},
                    {"match_phrase": {"Text": "ruby"}},
                    {"match_phrase": {"Text": "php"}}
                ]
            }
        },
        "aggs": {
            "Programming_Language": {
                "filters": {
                    "filters": {
                        "SQL": {"match_phrase": {"Text": "sql"}},
                        "Python": {"match_phrase": {"Text": "python"}},
                        "R": {"match_phrase": {"Text": " R "}},
                        "Java": {"match_phrase": {"Text": "java"}},
                        "JavaScript": {"match_phrase": {"Text": "javascript"}},
                        "C++": {"match_phrase": {"Text": "c++"}},
                        "Ruby": {"match_phrase": {"Text": "ruby"}},
                        "PHP": {"match_phrase": {"Text": "php"}}
                    }
                }
            }
        }
    }
)
execution_time = time.time() - start_time


### Query 15

Count how many users wrote sql in their about me and wrote a comment with the word python. If they also have the word c++ in the comment rank it higher. Order by count and weight (count of comment weight uses of c++)

```
SELECT userid, count(*) as total, WEIGHT() 
FROM usersjoincomments
WHERE MATCH('@AboutMe sql @text python MAYBE c++')  
GROUP BY userid
ORDER BY total DESC, WEIGHT() DESC

```
